In [17]:
pip install soundfile


Note: you may need to restart the kernel to use updated packages.


In [5]:
# -----------------------------
# 1. Install dependencies (if not installed)
# -----------------------------
# pip install TTS librosa soundfile ipython

import os
import librosa
import numpy as np
import soundfile as sf
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer
from IPython.display import Audio

# -----------------------------
# 2. Download / load TTS model
# -----------------------------
model_name = "tts_models/multilingual/multi-dataset/your_tts"
manager = ModelManager()
result = manager.download_model(model_name)

# Extract .pth and .json files
model_path = None
config_path = None

if isinstance(result, (tuple, list)):
    for item in result:
        if isinstance(item, str) and item.endswith(".pth"):
            model_path = item
        elif isinstance(item, str) and item.endswith(".json"):
            config_path = item
else:
    if isinstance(result, str):
        if result.endswith(".pth"):
            model_path = result
        elif result.endswith(".json"):
            config_path = result

# fallback: search folder for config
if config_path is None and model_path is not None:
    folder = os.path.dirname(model_path)
    for file in os.listdir(folder):
        if file.endswith(".json"):
            config_path = os.path.join(folder, file)
            break

if model_path is None or config_path is None:
    raise RuntimeError("Model checkpoint or config not found.")

print("Model checkpoint:", model_path)
print("Config path:", config_path)

# Initialize synthesizer
synthesizer = Synthesizer(tts_checkpoint=model_path, tts_config_path=config_path)

# -----------------------------
# 3. Preprocess parent voice
# -----------------------------
def preprocess_parent_voice(input_path, output_path="parent_voice_clean_16k.wav", sr=16000):
    """
    Load, trim silence, apply spectral gating noise reduction, normalize, and save 16kHz WAV.
    """
    # Load audio
    y, sr = librosa.load(input_path, sr=sr)

    # Trim silence
    y_trimmed, _ = librosa.effects.trim(y, top_db=20)

    # Simple spectral gating noise reduction
    def spectral_gate_denoise(y, sr):
        stft = librosa.stft(y)
        magnitude, phase = librosa.magphase(stft)
        noise_est = np.median(magnitude, axis=1, keepdims=True)
        mask = magnitude >= noise_est
        stft_clean = stft * mask
        return librosa.istft(stft_clean)

    y_denoised = spectral_gate_denoise(y_trimmed, sr)

    # Normalize
    y_normalized = librosa.util.normalize(y_denoised)

    # Save
    sf.write(output_path, y_normalized, sr)
    return output_path

input_voice_path = "/Users/ravindudinuththara/Desktop/Sliit/Parent_voice_clonning/parent_voice_16k.wav"
clean_voice_path = preprocess_parent_voice(input_voice_path)

print("Clean parent voice saved to:", clean_voice_path)

# -----------------------------
# 4. Generate TTS audio
# -----------------------------
text = "Hello baby, mommy is here. Everything is okay, stop crying."

audio = synthesizer.tts(
    text=text,
    speaker_wav=clean_voice_path,
    language_name="en"
)

# Save output
output_file = "/Users/ravindudinuththara/Desktop/Sliit/output_voice3.wav"
sf.write(output_file, audio, 16050)

print("Synthesized audio saved to:", output_file)

# -----------------------------
# 5. Play audio in Jupyter / IPython
# -----------------------------
Audio(filename=output_file)

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
Model checkpoint: /Users/ravindudinuththara/Library/Application Support/tts/tts_models--multilingual--multi-dataset--your_tts/model_file.pth
Config path: /Users/ravindudinuththara/Library/Application Support/tts/tts_models--multilingual--multi-dataset--your_tts/config.json
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | >

In [ ]:
# ===============================
# 1. Imports and Setup
# ===============================
import os
import random
import time
from collections import deque, defaultdict
from pprint import pprint

import numpy as np
import sounddevice as sd
import librosa
import pygame
from tensorflow.keras.models import load_model
import pickle
import soundfile as sf
from IPython.display import Audio

# --- LLM-based parental soothing ---
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# --- TTS ---
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer

# -------------------------------
# Audio and cry classification
# -------------------------------
SAMPLE_RATE = 16000
N_MFCC = 40
MAX_LEN = 216
SEGMENT_DURATION = 10  # seconds
SEGMENT_SIZE = SAMPLE_RATE * SEGMENT_DURATION
audio_buffer = deque(maxlen=SEGMENT_SIZE)

# Load Cry Model
model_path = '/Users/ravindudinuththara/Desktop/Research/Cry_Classification/cry_lstm_model00.h5'
model = load_model(model_path)
categories = ['belly pain', 'burping', 'discomfort', 'hungry', 'laugh',
              'lonely', 'noise', 'scared', 'silence', 'tired']
print("✅ Cry model loaded!")

# Music Base Dir & Emotion Mapping
MUSIC_BASE_DIR = "/Users/ravindudinuththara/Desktop/Research/Lullabies/categorized_music"
EMOTION_TO_CATEGORY = {
    "belly pain": "loud_high_pitched",
    "burping": "rhythmic_rising_pitch",
    "discomfort": "medium_pitch_whiny",
    "hungry": "rhythmic_rising_pitch",
    "laugh": "playful_upbeat",
    "lonely": "warm_soft_comforting",
    "noise": "neutral_noise_blocker",
    "scared": "trembling_pitch",
    "silence": "baseline_state",
    "tired": "low_energy_whining"
}

# Initialize Pygame mixer
pygame.mixer.init()

# ===============================
# 2. Feature Extraction & Prediction
# ===============================
def extract_features_from_audio(audio, sr=SAMPLE_RATE, n_mfcc=N_MFCC, max_len=MAX_LEN):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T
    if mfcc.shape[0] < max_len:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0,0)), mode='constant')
    else:
        mfcc = mfcc[:max_len, :]
    return np.expand_dims(mfcc, axis=0)

def predict_cry(audio):
    try:
        features = extract_features_from_audio(audio)
        pred = model.predict(features, verbose=0)
        idx = np.argmax(pred)
        return categories[idx], pred[0][idx]
    except Exception as e:
        print("Prediction error:", e)
        return None, 0.0

# Audio callback for real-time recording
def audio_callback(indata, frames, time_info, status):
    if status:
        print(status)
    audio_buffer.extend(indata[:,0])

# ===============================
# 3. Q-Learning Agent
# ===============================
class QLearningAgent:
    def __init__(self, states, actions, alpha=0.6, gamma=0.9, epsilon=0.2):
        self.states = states
        self.actions = actions
        self.Q = defaultdict(lambda: {a: 0.0 for a in self.actions})
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def choose_action(self, state):
        _ = self.Q[state]
        if random.random() < self.epsilon:
            print(f"🤖 RL Decision: Random action for state '{state}'")
            return random.choice(self.actions)
        qvals = self.Q[state]
        max_val = max(qvals.values())
        best_actions = [a for a, v in qvals.items() if v == max_val]
        action = random.choice(best_actions)
        print(f"🤖 RL Decision: Best action for state '{state}' -> '{action}'")
        return action

    def update(self, state, action, reward, next_state):
        _ = self.Q[state]
        _ = self.Q[next_state]
        q_sa = self.Q[state][action]
        max_next = max(self.Q[next_state].values()) if self.Q[next_state] else 0.0
        self.Q[state][action] = q_sa + self.alpha * (reward + self.gamma * max_next - q_sa)
        print(f"📝 Updated Q[{state}][{action}] = {self.Q[state][action]:.2f}")

    def save(self, filepath="q_table.pkl"):
        with open(filepath, "wb") as f:
            pickle.dump(dict(self.Q), f)
        print(f"💾 Q-table saved to {filepath}")

    def load(self, filepath="q_table.pkl"):
        try:
            with open(filepath, "rb") as f:
                qdict = pickle.load(f)
            self.Q = defaultdict(lambda: {a: 0.0 for a in self.actions}, qdict)
            print("✅ Loaded previous Q-table.")
        except FileNotFoundError:
            print("⚠️ No previous Q-table found. Starting fresh.")

# ===============================
# 4. LLM + TTS-based Parent Soother
# ===============================
class SimpleSoother:
    def __init__(self, model_name="gemma3:4b",
                 tts_model_name="tts_models/multilingual/multi-dataset/your_tts",
                 parent_name="Parent",
                 parent_voice_path=None):
        self.parent_name = parent_name
        self.parent_voice_path = parent_voice_path

        # --- Initialize LLM ---
        try:
            self.llm = ChatOllama(model=model_name, temperature=0.7)
            self.prompt_template = ChatPromptTemplate.from_messages([
                ("system", f"You are a loving parent named {self.parent_name}. "
                           "Your infant is crying. Generate a very short soothing word or phrase."),
            ])
            print(f"🟢 LLM model loaded: {model_name}")
        except Exception as e:
            print(f"❌ Failed to load LLM model '{model_name}': {e}")
            self.llm = None

        # --- Initialize TTS ---
        try:
            manager = ModelManager()
            result = manager.download_model(tts_model_name)
            model_path = None
            config_path = None
            if isinstance(result, (tuple, list)):
                for item in result:
                    if isinstance(item, str) and item.endswith(".pth"):
                        model_path = item
                    elif isinstance(item, str) and item.endswith(".json"):
                        config_path = item
            else:
                if isinstance(result, str):
                    if result.endswith(".pth"):
                        model_path = result
                    elif result.endswith(".json"):
                        config_path = item
            # fallback search
            if config_path is None and model_path:
                folder = os.path.dirname(model_path)
                for file in os.listdir(folder):
                    if file.endswith(".json"):
                        config_path = os.path.join(folder, file)
                        break
            if model_path is None or config_path is None:
                raise RuntimeError("TTS model checkpoint or config not found.")
            self.synthesizer = Synthesizer(tts_checkpoint=model_path, tts_config_path=config_path)
            print(f"🟢 TTS model loaded: {tts_model_name}")
        except Exception as e:
            print(f"❌ Failed to load TTS model '{tts_model_name}': {e}")
            self.synthesizer = None

    # --- Generate soothing phrase ---
    def get_soothing_phrase(self, emotion):
        if not self.llm:
            return "⚠️ No model loaded, cannot generate response."
        chain = self.prompt_template | self.llm
        response = chain.invoke({"human": f"The baby is feeling: {emotion}"})
        return response.content

    # --- Preprocess parent voice ---
    @staticmethod
    def preprocess_parent_voice(input_path, output_path="parent_voice_clean_16k.wav", sr=16000):
        y, sr = librosa.load(input_path, sr=sr)
        y_trimmed, _ = librosa.effects.trim(y, top_db=20)
        def spectral_gate_denoise(y, sr):
            stft = librosa.stft(y)
            magnitude, phase = librosa.magphase(stft)
            noise_est = np.median(magnitude, axis=1, keepdims=True)
            mask = magnitude >= noise_est
            stft_clean = stft * mask
            return librosa.istft(stft_clean)
        y_denoised = spectral_gate_denoise(y_trimmed, sr)
        y_normalized = librosa.util.normalize(y_denoised)
        sf.write(output_path, y_normalized, sr)
        return output_path

    # --- Generate TTS audio ---
    def speak(self, text, output_file="output_parent_voice.wav"):
        if not self.synthesizer:
            print("⚠️ No TTS synthesizer loaded, cannot speak.")
            return
        speaker_wav = None
        if self.parent_voice_path:
            speaker_wav = self.preprocess_parent_voice(self.parent_voice_path)
        audio = self.synthesizer.tts(text=text, speaker_wav=speaker_wav, language_name="en")
        sf.write(output_file, audio, 16000)
        print(f"🎙️ Synthesized audio saved to: {output_file}")
        return Audio(filename=output_file)

    # --- Full soothe + speak ---
    def soothe(self, emotion):
        phrase = self.get_soothing_phrase(emotion)
        print(f"👶 Baby emotion: {emotion} -> 🗣️ Parent says: '{phrase}'")
        return self.speak(phrase)

# Initialize soother
input_voice_path = "/Users/ravindudinuththara/Desktop/Sliit/Parent_voice_clonning/parent_voice_16k.wav"
soother = SimpleSoother(model_name="gemma3:4b",
                        tts_model_name="tts_models/multilingual/multi-dataset/your_tts",
                        parent_name="Dad",
                        parent_voice_path=input_voice_path)

# ===============================
# 5. Actions: Voice & Music
# ===============================
def parental_voice_cloning(state):
    print("🎙️ Starting parental voice soothing...")
    soother.soothe(state)
    print("🎙️ Finished parental voice soothing.")

def play_music_for_emotion(baby_emotion):
    print("🎵 Starting music playback...")
    if not baby_emotion:
        print("⚠️ No emotion provided")
        return
    emotion_key = baby_emotion.lower().strip()
    category = EMOTION_TO_CATEGORY.get(emotion_key, "baseline_state")
    category_folder = os.path.join(MUSIC_BASE_DIR, category)
    if not os.path.exists(category_folder):
        print(f"⚠️ Category folder not found: {category_folder}")
        return
    songs = [f for f in os.listdir(category_folder) if f.lower().endswith(('.mp3', '.wav'))]
    if not songs:
        print(f"⚠️ No songs found in {category_folder}")
        return
    song_file = random.choice(songs)
    song_path = os.path.join(category_folder, song_file)
    print(f"🎵 Playing: {song_file} (Emotion: {baby_emotion}, Category: {category})")
    pygame.mixer.music.load(song_path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(1)
    print("🎵 Music playback finished.")

def music_recommendation(state):
    play_music_for_emotion(state)

# ===============================
# 6. Reward & Next State
# ===============================
def get_reward_and_next_state(prev_state, action):
    if len(audio_buffer) < SEGMENT_SIZE:
        print("⚠️ Not enough audio for evaluation.")
        return prev_state, -1
    audio_segment = np.array(audio_buffer)
    predicted_class, confidence = predict_cry(audio_segment)
    if predicted_class is None:
        return prev_state, -1
    reward = 10 if predicted_class == "silence" else (-1 if predicted_class == prev_state else 1)
    print(f"🎯 Evaluated reward: {reward} (Next state predicted: {predicted_class})")
    return predicted_class, reward

# ===============================
# 7. Interaction Loop
# ===============================
def interact_with_real_baby(agent, max_steps=3):
    if len(audio_buffer) < SEGMENT_SIZE:
        print("⏳ Waiting for initial audio...")
        return
    print("🔊 Recording audio segment for prediction...")
    audio_segment = np.array(audio_buffer)
    state, conf = predict_cry(audio_segment)
    print(f"Initial baby state: {state} ({conf*100:.2f}%)")
    steps = 0

    while state != "silence" and steps < max_steps:
        print(f"🧠 RL Environment making decision... Step {steps+1}")
        action = agent.choose_action(state)
        print(f"🏷️ Chosen action: {action}")
        if action == "voice":
            parental_voice_cloning(state)
        else:
            music_recommendation(state)

        next_state, reward = get_reward_and_next_state(state, action)
        agent.update(state, action, reward, next_state)

        # Dynamic switching if reward is low
        if reward <= 0 and steps < max_steps - 1:
            other_action = [a for a in agent.actions if a != action][0]
            print(f"🔄 Reward low, trying switch to {other_action}")
            if other_action == "voice":
                parental_voice_cloning(state)
            else:
                music_recommendation(state)
            ns2, r2 = get_reward_and_next_state(state, other_action)
            agent.update(state, other_action, r2, ns2)
            state = ns2
        else:
            state = next_state
        steps += 1

# ===============================
# 8. Main Loop in Jupyter
# ===============================
states = categories
actions = ["voice", "music"]
agent = QLearningAgent(states, actions)
agent.load("/Users/ravindudinuththara/Desktop/Research/RLEngine/q_table.pkl")

print("🚀 Starting real-time baby soothing system in Jupyter. Press Ctrl+C to stop.")

# Start audio stream
stream = sd.InputStream(channels=1, samplerate=SAMPLE_RATE, callback=audio_callback, blocksize=1024)
stream.start()

try:
    while True:
        print("\n⏱️ New interaction cycle starting...")
        time.sleep(60)  # check every 1 minute
        interact_with_real_baby(agent)
        agent.save("/Users/ravindudinuththara/Desktop/Research/RLEngine/q_table.pkl")
except KeyboardInterrupt:
    print("🛑 Stopping system...")
finally:
    stream.stop()
    stream.close()

/opt/anaconda3/envs/researchenv/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


/opt/anaconda3/envs/researchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 13:23:15.682053: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-12-04 13:23:15.682209: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-12-04 13:23:15.682251: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1764834795.682697 2629282 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1764834795.682863 2629282 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


✅ Cry model loaded!
🟢 LLM model loaded: gemma3:4b
 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_

2025-12-04 13:24:17.628862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Initial baby state: scared (95.46%)
🧠 RL Environment making decision... Step 1
🤖 RL Decision: Best action for state 'scared' -> 'music'
🏷️ Chosen action: music
🎵 Starting music playback...
🎵 Playing: little-star-lullaby-417636.mp3 (Emotion: scared, Category: trembling_pitch)
🎵 Music playback finished.
🎯 Evaluated reward: -1 (Next state predicted: scared)
📝 Updated Q[scared][music] = -1.69
🔄 Reward low, trying switch to voice
🎙️ Starting parental voice soothing...
👶 Baby emotion: scared -> 🗣️ Parent says: '“Shhh, sweetie, it’s okay. Daddy’s here.” 

❤️'
 > Text splitted to sentences.
['“Shhh, sweetie, it’s okay. Daddy’s here.”', '❤️']
“shhh, sweetie, it’s okay. daddy’s here.”
 [!] Character '“' not found in the vocabulary. Discarding it.
“shhh, sweetie, it’s okay. daddy’s here.”
 [!] Character '’' not found in the vocabulary. Discarding it.
“shhh, sweetie, it’s okay. daddy’s here.”
 [!] Character '”' not found in the vocabulary. Discarding it.
❤️
 [!] Character '❤' not found in the voca

In [1]:
import tensorflow as tf
import numpy as np
import keras

print("TF:", tf.__version__)   # 2.19.0
print("Keras:", keras.__version__)  # 3.12.0
print("NumPy:", np.__version__)  # 1.26.0

TF: 2.19.0
Keras: 3.12.0
NumPy: 1.26.0


In [ ]:
# ===============================
# 1. Imports and Setup
# ===============================
import os
import random
import time
from collections import deque, defaultdict

import numpy as np
import sounddevice as sd
import librosa
import pygame
from tensorflow.keras.models import load_model
import pickle
import soundfile as sf

# --- LLM-based parental soothing ---
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# --- TTS ---
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer

# -------------------------------
# Audio and cry classification
# -------------------------------
SAMPLE_RATE = 16000
N_MFCC = 40
MAX_LEN = 216
SEGMENT_DURATION = 10  # seconds
SEGMENT_SIZE = SAMPLE_RATE * SEGMENT_DURATION
audio_buffer = deque(maxlen=SEGMENT_SIZE)

# Load Cry Model
model_path = '/Users/ravindudinuththara/Desktop/Research/Cry_Classification/cry_lstm_model00.h5'
model = load_model(model_path)
categories = ['belly pain', 'burping', 'discomfort', 'hungry', 'laugh',
              'lonely', 'noise', 'scared', 'silence', 'tired']
print("✅ Cry model loaded!")

# Music Base Dir & Emotion Mapping
MUSIC_BASE_DIR = "/Users/ravindudinuththara/Desktop/Research/Lullabies/categorized_music"
EMOTION_TO_CATEGORY = {
    "belly pain": "loud_high_pitched",
    "burping": "rhythmic_rising_pitch",
    "discomfort": "medium_pitch_whiny",
    "hungry": "rhythmic_rising_pitch",
    "laugh": "playful_upbeat",
    "lonely": "warm_soft_comforting",
    "noise": "neutral_noise_blocker",
    "scared": "trembling_pitch",
    "silence": "baseline_state",
    "tired": "low_energy_whining"
}

# Initialize Pygame mixer
pygame.mixer.init()

# ===============================
# 2. Feature Extraction & Prediction
# ===============================
def extract_features_from_audio(audio, sr=SAMPLE_RATE, n_mfcc=N_MFCC, max_len=MAX_LEN):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    mfcc = mfcc.T
    if mfcc.shape[0] < max_len:
        pad_width = max_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0, pad_width), (0,0)), mode='constant')
    else:
        mfcc = mfcc[:max_len, :]
    return np.expand_dims(mfcc, axis=0)

def predict_cry(audio):
    try:
        features = extract_features_from_audio(audio)
        pred = model.predict(features, verbose=0)
        idx = np.argmax(pred)
        return categories[idx], pred[0][idx]
    except Exception as e:
        print("Prediction error:", e)
        return None, 0.0

# Audio callback for real-time recording
def audio_callback(indata, frames, time_info, status):
    if status:
        print(statu)
    audio_buffer.extend(indata[:,0])

# ===============================
# 3. Q-Learning Agent
# ===============================
class QLearningAgent:
    def __init__(self, states, actions, alpha=0.6, gamma=0.9, epsilon=0.2):
        self.states = states
        self.actions = actions
        self.Q = defaultdict(lambda: {a: 0.0 for a in self.actions})
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def choose_action(self, state):
        _ = self.Q[state]
        if random.random() < self.epsilon:
            print(f"🤖 RL Decision: Random action for state '{state}'")
            return random.choice(self.actions)
        qvals = self.Q[state]
        max_val = max(qvals.values())
        best_actions = [a for a, v in qvals.items() if v == max_val]
        action = random.choice(best_actions)
        print(f"🤖 RL Decision: Best action for state '{state}' -> '{action}'")
        return action

    def update(self, state, action, reward, next_state):
        _ = self.Q[state]
        _ = self.Q[next_state]
        q_sa = self.Q[state][action]
        max_next = max(self.Q[next_state].values()) if self.Q[next_state] else 0.0
        self.Q[state][action] = q_sa + self.alpha * (reward + self.gamma * max_next - q_sa)
        print(f"📝 Updated Q[{state}][{action}] = {self.Q[state][action]:.2f}")

    def save(self, filepath="q_table.pkl"):
        with open(filepath, "wb") as f:
            pickle.dump(dict(self.Q), f)
        print(f"💾 Q-table saved to {filepath}")

    def load(self, filepath="q_table.pkl"):
        try:
            with open(filepath, "rb") as f:
                qdict = pickle.load(f)
            self.Q = defaultdict(lambda: {a: 0.0 for a in self.actions}, qdict)
            print("✅ Loaded previous Q-table.")
        except FileNotFoundError:
            print("⚠️ No previous Q-table found. Starting fresh.")

# ===============================
# 4. LLM + TTS-based Parent Soother
# ===============================
class SimpleSoother:
    def __init__(self, model_name="gemma3:4b",
                 tts_model_name="tts_models/multilingual/multi-dataset/your_tts",
                 parent_name="Parent",
                 parent_voice_path=None):
        self.parent_name = parent_name
        self.parent_voice_path = parent_voice_path

        # --- Initialize LLM ---
        try:
            self.llm = ChatOllama(model=model_name, temperature=0.7)
            self.prompt_template = ChatPromptTemplate.from_messages([
                ("system", f"You are a loving parent named {self.parent_name}. "
                           "Your infant is crying. Generate a very short soothing word or phrase."),
            ])
            print(f"🟢 LLM model loaded: {model_name}")
        except Exception as e:
            print(f"❌ Failed to load LLM model '{model_name}': {e}")
            self.llm = None

        # --- Initialize TTS ---
        try:
            manager = ModelManager()
            result = manager.download_model(tts_model_name)
            model_path = None
            config_path = None
            if isinstance(result, (tuple, list)):
                for item in result:
                    if isinstance(item, str) and item.endswith(".pth"):
                        model_path = item
                    elif isinstance(item, str) and item.endswith(".json"):
                        config_path = item
            else:
                if isinstance(result, str):
                    if result.endswith(".pth"):
                        model_path = result
                    elif result.endswith(".json"):
                        config_path = item
            # fallback search
            if config_path is None and model_path:
                folder = os.path.dirname(model_path)
                for file in os.listdir(folder):
                    if file.endswith(".json"):
                        config_path = os.path.join(folder, file)
                        break
            if model_path is None or config_path is None:
                raise RuntimeError("TTS model checkpoint or config not found.")
            self.synthesizer = Synthesizer(tts_checkpoint=model_path, tts_config_path=config_path)
            print(f"🟢 TTS model loaded: {tts_model_name}")
        except Exception as e:
            print(f"❌ Failed to load TTS model '{tts_model_name}': {e}")
            self.synthesizer = None

    # --- Generate soothing phrase ---
    def get_soothing_phrase(self, emotion):
        if not self.llm:
            return "⚠️ No model loaded, cannot generate response."
        chain = self.prompt_template | self.llm
        response = chain.invoke({"human": f"The baby is feeling: {emotion}"})
        return response.content

    # --- Preprocess parent voice ---
    @staticmethod
    def preprocess_parent_voice(input_path, output_path="parent_voice_clean_16k.wav", sr=16000):
        y, sr = librosa.load(input_path, sr=sr)
        y_trimmed, _ = librosa.effects.trim(y, top_db=20)
        def spectral_gate_denoise(y, sr):
            stft = librosa.stft(y)
            magnitude, phase = librosa.magphase(stft)
            noise_est = np.median(magnitude, axis=1, keepdims=True)
            mask = magnitude >= noise_est
            stft_clean = stft * mask
            return librosa.istft(stft_clean)
        y_denoised = spectral_gate_denoise(y_trimmed, sr)
        y_normalized = librosa.util.normalize(y_denoised)
        sf.write(output_path, y_normalized, sr)
        return output_path

    # --- Generate TTS audio and play automatically ---
    def speak(self, text, output_file="output_parent_voice.wav"):
        if not self.synthesizer:
            print("⚠️ No TTS synthesizer loaded, cannot speak.")
            return
        speaker_wav = None
        if self.parent_voice_path:
            speaker_wav = self.preprocess_parent_voice(self.parent_voice_path)
        audio = self.synthesizer.tts(text=text, speaker_wav=speaker_wav, language_name="en")
        sf.write(output_file, audio, 16000)
        print(f"🎙️ Synthesized audio saved to: {output_file}")

        # Play the audio automatically using pygame
        try:
            pygame.mixer.init()
            pygame.mixer.music.load(output_file)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                time.sleep(0.1)
        except Exception as e:
            print("⚠️ Could not play audio automatically:", e)

    # --- Full soothe + speak ---
    def soothe(self, emotion):
        phrase = self.get_soothing_phrase(emotion)
        print(f"👶 Baby emotion: {emotion} -> 🗣️ Parent says: '{phrase}'")
        return self.speak(phrase)

# Initialize soother
input_voice_path = "/Users/ravindudinuththara/Desktop/Sliit/Parent_voice_clonning/parent_voice_16k.wav"
soother = SimpleSoother(model_name="gemma3:4b",
                        tts_model_name="tts_models/multilingual/multi-dataset/your_tts",
                        parent_name="Dad",
                        parent_voice_path=input_voice_path)

# ===============================
# 5. Actions: Voice & Music
# ===============================
def parental_voice_cloning(state):
    print("🎙️ Starting parental voice soothing...")
    soother.soothe(state)
    print("🎙️ Finished parental voice soothing.")

def play_music_for_emotion(baby_emotion):
    print("🎵 Starting music playback...")
    if not baby_emotion:
        print("⚠️ No emotion provided")
        return
    emotion_key = baby_emotion.lower().strip()
    category = EMOTION_TO_CATEGORY.get(emotion_key, "baseline_state")
    category_folder = os.path.join(MUSIC_BASE_DIR, category)
    if not os.path.exists(category_folder):
        print(f"⚠️ Category folder not found: {category_folder}")
        return
    songs = [f for f in os.listdir(category_folder) if f.lower().endswith(('.mp3', '.wav'))]
    if not songs:
        print(f"⚠️ No songs found in {category_folder}")
        return
    song_file = random.choice(songs)
    song_path = os.path.join(category_folder, song_file)
    print(f"🎵 Playing: {song_file} (Emotion: {baby_emotion}, Category: {category})")
    pygame.mixer.music.load(song_path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        time.sleep(1)
    print("🎵 Music playback finished.")

def music_recommendation(state):
    play_music_for_emotion(state)

# ===============================
# 6. Reward & Next State
# ===============================
def get_reward_and_next_state(prev_state, action):
    if len(audio_buffer) < SEGMENT_SIZE:
        print("⚠️ Not enough audio for evaluation.")
        return prev_state, -1
    audio_segment = np.array(audio_buffer)
    predicted_class, confidence = predict_cry(audio_segment)
    if predicted_class is None:
        return prev_state, -1
    reward = 10 if predicted_class == "silence" else (-1 if predicted_class == prev_state else 1)
    print(f"🎯 Evaluated reward: {reward} (Next state predicted: {predicted_class})")
    return predicted_class, reward

# ===============================
# 7. Interaction Loop
# ===============================
def interact_with_real_baby(agent, max_steps=3):
    if len(audio_buffer) < SEGMENT_SIZE:
        print("⏳ Waiting for initial audio...")
        return
    print("🔊 Recording audio segment for prediction...")
    audio_segment = np.array(audio_buffer)
    state, conf = predict_cry(audio_segment)
    print(f"Initial baby state: {state} ({conf*100:.2f}%)")
    steps = 0

    while state != "silence" and steps < max_steps:
        print(f"🧠 RL Environment making decision... Step {steps+1}")
        action = agent.choose_action(state)
        print(f"🏷️ Chosen action: {action}")
        if action == "voice":
            parental_voice_cloning(state)
        else:
            music_recommendation(state)

        next_state, reward = get_reward_and_next_state(state, action)
        agent.update(state, action, reward, next_state)

        # Dynamic switching if reward is low
        if reward <= 0 and steps < max_steps - 1:
            other_action = [a for a in agent.actions if a != action][0]
            print(f"🔄 Reward low, trying switch to {other_action}")
            if other_action == "voice":
                parental_voice_cloning(state)
            else:
                music_recommendation(state)
            ns2, r2 = get_reward_and_next_state(state, other_action)
            agent.update(state, other_action, r2, ns2)
            state = ns2
        else:
            state = next_state
        steps += 1

# ===============================
# 8. Main Loop in Jupyter
# ===============================
states = categories
actions = ["voice", "music"]
agent = QLearningAgent(states, actions)
agent.load("/Users/ravindudinuththara/Desktop/Research/RLEngine/q_table.pkl")

print("🚀 Starting real-time baby soothing system in Jupyter. Press Ctrl+C to stop.")

# Start audio stream
stream = sd.InputStream(channels=1, samplerate=SAMPLE_RATE, callback=audio_callback, blocksize=1024)
stream.start()

try:
    while True:
        print("\n⏱️ New interaction cycle starting...")
        time.sleep(60)  # check every 1 minute
        interact_with_real_baby(agent)
        agent.save("/Users/ravindudinuththara/Desktop/Research/RLEngine/q_table.pkl")
except KeyboardInterrupt:
    print("🛑 Stopping system...")
finally:
    stream.stop()
    stream.close()

/opt/anaconda3/envs/researchenv/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


/opt/anaconda3/envs/researchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-04 13:35:30.752842: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-12-04 13:35:30.752980: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-12-04 13:35:30.752997: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
I0000 00:00:1764835530.753352 2644470 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1764835530.754198 2644470 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


✅ Cry model loaded!
🟢 LLM model loaded: gemma3:4b
 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_

2025-12-04 13:36:32.924136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Initial baby state: scared (94.02%)
🧠 RL Environment making decision... Step 1
🤖 RL Decision: Best action for state 'scared' -> 'music'
🏷️ Chosen action: music
🎵 Starting music playback...
🎵 Playing: creeping-music-box-horror-lullaby-365186.mp3 (Emotion: scared, Category: trembling_pitch)
🎵 Music playback finished.
🎯 Evaluated reward: -1 (Next state predicted: scared)
📝 Updated Q[scared][music] = -1.69
🔄 Reward low, trying switch to voice
🎙️ Starting parental voice soothing...
👶 Baby emotion: scared -> 🗣️ Parent says: '“Shhh, sweetie, it’s okay.” 

(I gently stroke their cheek)'
 > Text splitted to sentences.
['“Shhh, sweetie, it’s okay.”', '(I gently stroke their cheek)']
“shhh, sweetie, it’s okay.”
 [!] Character '“' not found in the vocabulary. Discarding it.
“shhh, sweetie, it’s okay.”
 [!] Character '’' not found in the vocabulary. Discarding it.
“shhh, sweetie, it’s okay.”
 [!] Character '”' not found in the vocabulary. Discarding it.
 > Processing time: 3.1031768321990967
 > Rea